# Page Fault Visualizer

This notebook enables visualizing the page fault behavior per file.


In [ ]:
%load_ext autoreload
%autoreload 2
import altair as alt
from ipywidgets import widgets
alt.data_transformers.enable("vegafusion")

from utilities import load_mappings, page_fault_chart,  time_based_page_fault_chart, extract_faults, fault_time_series


In [ ]:
# Replace with `output` to use locally generated files instead of example
mapped_faults, file_sizes = load_mappings("example")

In [ ]:
file_names = list(set(m["file_name"] for m in mapped_faults))
file_name_widget = widgets.Dropdown(
    options=file_names,
    description="File Name:",
    layout=widgets.Layout(padding="0px 0px 200px 0px", width="600px"),
    disabled=False,
)

display(file_name_widget)


# Param: If applicable, the name of file embedded within APK
apk_entry_name = None

In [ ]:
faults, _, _ = extract_faults(
    file_name_widget.value, apk_entry_name, file_sizes, mapped_faults
)
fault_time_series(faults)

In [ ]:
page_fault_chart(file_name_widget.value, apk_entry_name, file_sizes, mapped_faults)

In [ ]:
time_based_page_fault_chart(
    file_name_widget.value, apk_entry_name, file_sizes, mapped_faults
)